In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from BlackScholes import BS_call
from Barrier import (volatility, SDE_downandout_put, SDE_downandout_put_ant,SDE_downandout_put_delta)

In this section we will we will simulate prices for an asset whos underlying spot price $S(t)$ follows Geometric Brownian Motion with local volatility $\sigma(S,t)$ which is a function of the spot price and time:

$$ dS(t) = rS(t)dt + \sigma(S_{t},t)S_{t}dW_{t} $$

$$ \sigma(S,t) = \sigma_0(1 + \sigma_1Cos(2\pi))(1+\sigma_2exp(-S/50))$$

where

$$\sigma_0 = 0.2$$
$$\sigma_1 = 0.3$$
$$\sigma_2 = 0.5$$





We will use this spot price and local volatility to price a certain type of barrier option known as a down-and-out put option. This added complexity of a changing volatility rate will closer replicate real world scenarios in which market forces such as volatility are not consant.

A down-and-out put option is a path dependent option whos payoff depends on if the value of underlying asset $S_t$ surpasses or does not surpass a specified barrier $S_b$ before expiry, given by the following payoff, with $K$ denoting the strike price of the option:

$$\begin{cases}
  max(K - S_{T},0), = \text {if  }  \min_t S_t > S_b\\    
  0,  \qquad \qquad \qquad \, \text {if }  \min_t S_t \leq S_b
\end{cases}$$

Note that if the barrier $S_b$ is equal to zero the payoff is exactly the same as a vanilla european put option.

We will price the option initally with the following parameters:
<br>
$K$ = £50 <br>
$T$ = 1 year <br>
$r$ = 0.05 <br>
$t$ = 260, 260 working days in a year. Where we Euler time step by 1 day. <br>
$N$ = 5000, the number of sample paths used when pricing $S(t))$

In [ ]:
#Setting Parameter Values

sigma_0 = 0.2
sigma_1 = 0.3
sigma_2 = 0.5

#S0 = 45
K = 50
T = 1
r = 0.05
t = 260
Sb = 30
Nsteps = int(t * T) 
Npaths = 5000 


In [ ]:
#Running naive montecarlo for asset prices from S = 30 to S = 100 in steps of 5 for an inital 5000 simulated paths
#paths for the stock price S(T)

# S_plot = array of stock prices ranging from 30 to 100 inclusive in steps of 5
S_plot = np.arange(30,100,1)

#Npts is equal to the number of stock prices we will calculate the price of the barrier option for
Npts = len(S_plot)

#Setting empty arrays for which the monte carlo prices and variances will be stored in for corresponding stock prices
MC = np.zeros(Npts)
var = np.zeros(Npts)

# Running 5000 simulations for each stock price in our S_plot array and calculating the price and variance of each.
for k in range(Npts):
    MC[k], var[k] =SDE_downandout_put(S_plot[k],Sb,K,T,r,Npaths,sigma_0,sigma_1,sigma_2)
    
#For illustation purposes we will plot the bull call spread prices with 5000 sims against S before doing anything else.
SE = np.sqrt(var/Npaths)
plt.plot(S_plot,MC,marker = 'o', linestyle = '-',color = 'r',alpha = 0.45,label = "Naive")
plt.xlabel("Spot price S")
plt.ylabel("Bull Call Spread Price")
plt.title("Plot of bull call spread price for naive MC method ")
plt.legend()
plt.show()
max_var = np.amax(var)

For illustration we will plot the 95% confidence interval for the barrier option using the naive MC method

In [ ]:
# Calculating the 95% confidence interval upper and lower bounds
intp = MC + 1.96*SE
intm = MC - 1.96*SE

#Plotting
plt.figure(figsize = (10,7))
plt.plot(S_plot,MC, label = "Naive")
plt.fill_between(S_plot,intp,intm, color = "darkorange",alpha = 0.5,label = "95% CI")
plt.xlabel("Spot price S")
plt.ylabel("Bull Call Spread Value")
plt.title("Plot of bull call spread and 95% CI of naive MC method ")
plt.legend()
plt.show()

### And now repeating using variance reduction using antithetic variance reduction.


In [ ]:
#Running naive montecarlo with antithetic for asset prices from S = 10 to S = 200 in steps of 5 for an inital 1000 simulations.
# N = number of simulations

# S_plot = array of stock prices ranging from 10 to 200 inclusive in steps of 5
S_plot = np.arange(30,100,1)

#Npts is equal to the number of stock prices we will calculate the price of the bull call spread for
Npts = len(S_plot)

#Setting empty arrays for which the monte carlo bull call prices variances will be stored in for corresponding stock prices
MCa = np.zeros(Npts)
vara = np.zeros(Npts)

# Running 1000 simulations for each stock price in our S_plot array and calculating the price and variance of each.
for k in range(Npts):
    MCa[k], vara[k] =SDE_downandout_put_ant(S_plot[k],Sb,K,T,r,Npaths,sigma_0,sigma_1,sigma_2)
    
#For illustation purposes we will plot the bull call spread prices with 1000sims against S before doing anything else.
SE = np.sqrt(vara/Npaths)
plt.plot(S_plot,MCa,marker = 'o', linestyle = '-',color = 'r',alpha = 0.45,label = "Antithetic")

plt.xlabel("Spot price S")
plt.ylabel("Bull Call Spread Value")
plt.title("Plot of bull call spread for Antithetic MC method ")
plt.legend()
plt.show()



In [ ]:
intpa = MCa + 1.96*SE
intma = MCa - 1.96*SE
plt.figure(figsize = (10,7))
plt.plot(S_plot,MCa, label = "Naive")
plt.fill_between(S_plot,intpa,intma, color = "darkorange",alpha = 0.5,label = "95% CI")
plt.xlabel("Spot price S")
plt.ylabel("Bull Call Spread Value")
plt.title("Plot of bull call spread price for Antithetic MC method ")
plt.ylim(0,6)
plt.legend()
plt.show()

#### We note how the shaded region indicating the 95% Confidence interval using antithetic variance reduction is smaller than for the naive method, indicating some variance reduction was achieved

#### Now we will investigate the effect of changing the value of the barrier on the option price.

In [ ]:
#Setting our list of new barriers
Sbs = [0, 7, 15, 30, 49]
m = len(Sbs)
q = np.arange(0,m,1)
S_plot = np.arange(0.1,100,1)

#Npts is equal to the number of stock prices we will calculate the price of the bull call spread for
Npts = len(S_plot)

#Setting empty arrays for which the monte carlo bull call prices variances will be stored in for corresponding stock prices
MC = np.zeros([Npts,m], dtype = 'object')
var = np.zeros([Npts,m],dtype = 'object')


#Calculating Option Prices for different barriers and plotting:
for i in q:
    Sb = Sbs[i]
    for k in range(Npts):
        MC[k,i], var[k,i]= SDE_downandout_put_ant(S_plot[k],Sb,K,T,r,Npaths,sigma_0,sigma_1,sigma_2)
#Plotting    
    plt.plot(S_plot,MC[:,i], label = 'Sb = ' + str(Sb) +'')
plt.xlabel("Spot price S")
plt.ylabel("Barrier Option Price")
plt.title("Plot of barrier option for Antithetic MC for varying barriers ")
plt.legend()
plt.show()

### Delta

-------
I will calculate the delta of the option using path recycling methods similar to those found in Part 1

In [ ]:
#Running naive montecarlo for asset prices from S = 30 to S = 100 in steps of 5 for an inital 5000 simulated paths
#paths for the stock price S(T)

# S_plot = array of stock prices ranging from 30 to 100 inclusive in steps of 5
Sbs = [7, 15, 30, 49]
m = len(Sbs)
q = np.arange(0,m,1)
S_plot = np.arange(0.1,100,1)

#Npts is equal to the number of stock prices we will calculate the price of the bull call spread for
Npts = len(S_plot)

#Setting empty arrays for which the monte carlo bull call prices variances will be stored in for corresponding stock prices
MC = np.zeros([Npts,m], dtype = 'object')
var = np.zeros([Npts,m],dtype = 'object')


#Calculating Option Delta for different barriers and plotting:
for i in q:
    Sb = Sbs[i]
    for k in range(Npts):
        MC[k,i], var[k,i]= SDE_downandout_put_delta(S_plot[k],Sb,K,T,r,Npaths,sigma_0,sigma_1,sigma_2)
    
    plt.plot(S_plot,MC[:,i], label = 'Sb = ' + str(Sb) +'')
plt.xlabel("Spot price S")
plt.ylabel("Barrier Option Spread Price")
plt.title("Plot of Barrier Option Delta for Varying Barriers ")
plt.legend()
plt.show()




### Disussion

The variance reduction achieved using antithetic variance reduction may not be good enough for pricing contracts to be used in a live market making setting as the variances of our prices can be a more than a few basis points, hence we would want a much lower variance by possibly having more iterations which could take more time to achieve, or trying different variance reduction techniques, such as importance sampling or stratified sampling.

We note that for calculating the price of the spread the largest variances occur around where the barrier and strike prices are, hence we would maybe like to increase the number of iterations or perhaps use a different kind of variance reduction. 


We note how the plots of of our delta are somewhat "rough" comapared to that of the option prices, so ideally we would simulate more paths when calculating the deltas, or consider a different but more complex method of calculating the delta such as liklihood ratio methods which can be used even if the payoff of the contract is discontinuous. The "roughness" in our delta comes from the fact that large changes in delta that can occur from small changes of the spot price especailly around the barrier value.

But we note finding the symbolic derivative of this payoff equation along with the evolution of the local volatility would be difficult if we did want to condsider different delta calculation methods.